# The Aim of this notebook is to generate porosity data from a non-square part.

To do this, we will: 
- Read the image in. 
- Find the largest feature. 
- Create a mask that of the largest feature. 
- Segment the porosity. 
- Calculate area fraction.

In [71]:
#import statements 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import os 
import numpy as np

from skimage.filters import difference_of_gaussians,gaussian
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops,regionprops_table ,find_contours
from skimage.color import label2rgb,rgb2gray
import matplotlib.patches as mpatches
import imageio.v3 as iio
from skimage.morphology import remove_small_holes, remove_small_objects, binary_dilation, binary_erosion
# Fill in the empty holes 
from scipy.ndimage import binary_fill_holes

In [15]:
def show_before_after(image_before, image_after, str_change, i_cmap):
    if i_cmap == 1:
        str_cmap = "gray"
    elif i_cmap == 2:
        str_cmap = "plasma"
    elif i_cmap == 3:
        str_cmap = "seismic"
    fig,axes = plt.subplots(1,2,figsize=(12,4))
    axes[0].imshow(image_before, cmap=str_cmap)
    axes[0].set_title("Before "+str_change)
    axes[1].imshow(image_after, cmap=str_cmap)
    axes[1].set_title("After "+str_change)


In [ ]:
test_img_file = '../data/Sample_data/20250129_AlSi_01.png'
os.path.exists(test_img_file)

True

### Clean the image to get a good outline

In [73]:
def percentage_porosity_largePart(file_name):    
# Read image
    img = plt.imread(test_img_file)
    image_grayscale = rgb2gray(img)

    #Otsu threhold to get greyscale image.
    thresh = threshold_otsu(image_grayscale)
    binary = image_grayscale > thresh

    # Remove artifacts from the image
    removed_small_objects = remove_small_objects(binary, 6,False)
    tidy_image = remove_small_holes(removed_small_objects, area_threshold=5)

    # label image regions
    label_image = label(tidy_image)

    regions = regionprops(label_image)

    # Find the region with the largest area
    if regions:
        largest_region = max(regions, key=lambda r: r.area)
        
        # Create a new mask containing only the largest object
        # The new mask will have 1s where the largest object is, and 0s elsewhere.
        largest_object_mask = (label_image == largest_region.label)
    else:
        print("No objects found in the image mask.")

    filled_mask = binary_fill_holes(largest_object_mask)

    holes_mask = filled_mask != largest_object_mask

    labeled_holes = label(holes_mask)

    props = regionprops_table(labeled_holes, properties=['label', 'area'])
    data = pd.DataFrame(props)  
    sum_pores_px = data.area.sum()
    Total_Area =np.count_nonzero(filled_mask)

    percentage_porosity = (1 - (sum_pores_px/Total_Area)) * 100
    return percentage_porosity

In [74]:
percentage_porosity_largePart(test_img_file)

91.38779462721176